In [2]:
import pandas as pd
import numpy as np

In [3]:
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url) #training set


In [4]:
test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url) #test set

In [5]:
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
train.tail()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [7]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
train["Age"] = train["Age"].fillna(train["Age"].median())
train.Age.median()

np.float64(28.0)

In [9]:
train.loc[train["Embarked"] == "S", "Embarked"] = 0
train.loc[train["Embarked"] == "C", "Embarked"] = 1
train.loc[train["Embarked"] == "Q", "Embarked"] = 2
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])
train.Embarked.mode()[0]

C:\Users\tyrot\AppData\Local\Temp\ipykernel_30252\1115688226.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])


np.int64(0)

In [10]:
train.loc[train["Sex"] == "male", "Sex"] = 0
train.loc[train["Sex"] == "female", "Sex"] = 1
train["Sex"] = train["Sex"].fillna(train["Sex"].mode()[0])
train.Sex.mode()[0]

C:\Users\tyrot\AppData\Local\Temp\ipykernel_30252\1593609598.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train["Sex"] = train["Sex"].fillna(train["Sex"].mode()[0])


np.int64(0)

In [ ]:
# --- เตรียมข้อมูล X และ y ---
# เลือก Feature (แก้ PClass -> Pclass ให้ตรงกับชื่อใน DataFrame)
features = ["Pclass", "Sex", "Age", "Embarked"]
X = train[features].values.astype(float)
y = train["Survived"].values.astype(float) # Target

# --- Feature Scaling (Normalization) ---
# จำเป็นมากสำหรับ Gradient Descent เพื่อให้ค่าทุกตัวอยู่ในช่วงใกล้เคียงกัน
# สูตร: (x - mean) / std
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_norm = (X - X_mean) / X_std

# --- เพิ่ม Bias Term (Intercept) ---
# เพิ่มคอลัมน์ที่มีค่า 1 ทั้งหมดเข้าไปข้างหน้า เพื่อเป็นตัวคูณกับ theta0
m = len(y)
X_b = np.c_[np.ones((m, 1)), X_norm] 

# --- Linear Regression using Gradient Descent ---
# Hyperparameters
learning_rate = 0.01
n_iterations = 2000

# Initialize theta (parameters) ด้วย 0
theta = np.zeros(X_b.shape[1])

# Gradient Descent Loop
for iteration in range(n_iterations):
    # 1. Hypothesis: h(x) = X * theta
    predictions = X_b.dot(theta)
    
    # 2. Loss (Error): predictions - y
    error = predictions - y
    
    # 3. Gradient Calculation
    gradients = (1/m) * X_b.T.dot(error)
    
    # 4. Update Theta
    theta = theta - learning_rate * gradients
    
    # (Optional) ปริ้น Cost ทุกๆ 500 รอบเพื่อดูว่ามันลดลงไหม
    if iteration % 500 == 0:
        cost = (1/(2*m)) * np.sum(error**2)
        print(f"Iteration {iteration}: Cost = {cost}")

print("------------------------------------------------")
print(f"Final Theta: {theta}")

# --- Prediction & Thresholding (Check Accuracy on Train set) ---
final_predictions = X_b.dot(theta)
# ตัด Threshold ที่ 0.5 ตามโจทย์
predicted_class = (final_predictions >= 0.5).astype(int)

# คำนวณความแม่นยำเบื้องต้น
accuracy = np.mean(predicted_class == y)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

Iteration 0: Cost = 0.19191919191919193
Iteration 500: Cost = 0.07247011930294904
Iteration 1000: Cost = 0.0724628765442555
Iteration 1500: Cost = 0.07246286885059024
------------------------------------------------
Final Theta: [ 0.38383838 -0.15746008  0.23449806 -0.06576902  0.03120252]
Training Accuracy: 79.91%


In [12]:
# --- T11: Generate Submission File ---

# 1. จัดการ Test Set (ต้องทำเหมือน Train เป๊ะๆ)
# เติม Age ที่หายไป (ใช้ median ของ Train)
test["Age"] = test["Age"].fillna(train["Age"].median())

# เติม Embarked (ถ้ามีหาย) และ Map เป็นตัวเลข
test["Embarked"] = test["Embarked"].fillna(train["Embarked"].mode()[0])
test.loc[test["Embarked"] == "S", "Embarked"] = 0
test.loc[test["Embarked"] == "C", "Embarked"] = 1
test.loc[test["Embarked"] == "Q", "Embarked"] = 2

# Map Sex เป็นตัวเลข
test.loc[test["Sex"] == "male", "Sex"] = 0
test.loc[test["Sex"] == "female", "Sex"] = 1

# 2. เตรียมข้อมูล X_test
# เลือก Feature เดิม: Pclass, Sex, Age, Embarked
X_test = test[["Pclass", "Sex", "Age", "Embarked"]].values.astype(float)

# 3. Feature Scaling (ใช้ Mean และ Std จาก Train Set เท่านั้น!)
# X_mean และ X_std คือตัวแปรที่เราสร้างไว้ตอนทำ Train (ข้อ T10)
X_test_norm = (X_test - X_mean) / X_std

# 4. เพิ่ม Bias Term (Intercept)
m_test = len(X_test)
X_test_b = np.c_[np.ones((m_test, 1)), X_test_norm]

# 5. ทำนายผล (Predict)
# ใช้ theta ที่ได้จากการเทรน (ข้อ T10)
test_predictions = X_test_b.dot(theta)

# 6. ตัด Threshold ที่ 0.5
# ถ้า >= 0.5 ให้เป็น 1 (รอด), น้อยกว่าเป็น 0 (ไม่รอด)
submission_predictions = (test_predictions >= 0.5).astype(int)

# 7. สร้าง DataFrame สำหรับ Submission
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": submission_predictions
})

# ตรวจดูหน้าตาข้อมูลก่อน save
print(submission.head())

# 8. Save เป็นไฟล์ CSV (ไม่ต้องมี index)
submission.to_csv("submission.csv", index=False)
print("Saved submission.csv successfully!")

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
Saved submission.csv successfully!


In [13]:
# --- T12: Higher Order Features (Polynomial Regression) ---

def create_poly_features(X_in):
    """
    สร้างฟีเจอร์ดีกรี 2 (x^2 และ interaction terms x_i * x_j)
    """
    X_out = X_in.copy()
    n_features = X_in.shape[1]
    
    # Loop สร้าง pair ของฟีเจอร์ทุกคู่ (รวมถึงตัวมันเองกำลังสอง)
    for i in range(n_features):
        for j in range(i, n_features):
            new_col = X_in[:, i] * X_in[:, j]
            new_col = new_col.reshape(-1, 1)
            X_out = np.hstack((X_out, new_col))
            
    return X_out

# 1. เตรียมข้อมูล Train ใหม่ (เพิ่ม Polynomial)
# ใช้ X ตัวเดิมจากข้อ T10 (ที่มี Pclass, Sex, Age, Embarked)
X_poly = create_poly_features(X)

print(f"Original features: {X.shape[1]}")
print(f"New Polynomial features: {X_poly.shape[1]}")

# 2. Feature Scaling (สำคัญมากกก สำหรับ Poly เพราะค่ายิ่งต่างกันเยอะ)
X_poly_mean = X_poly.mean(axis=0)
X_poly_std = X_poly.std(axis=0)
X_poly_norm = (X_poly - X_poly_mean) / X_poly_std

# 3. Add Bias
m = len(y)
X_poly_b = np.c_[np.ones((m, 1)), X_poly_norm]

# 4. Train Model (Gradient Descent)
# อาจต้องปรับ learning rate หรือ n_iterations ถ้า cost ไม่ลง
learning_rate = 0.01 # ลองปรับเป็น 0.03 หรือ 0.1 ถ้ามันเรียนรู้ช้า
n_iterations = 5000  # เพิ่มรอบเพราะฟีเจอร์เยอะขึ้น

theta_poly = np.zeros(X_poly_b.shape[1])

for iteration in range(n_iterations):
    predictions = X_poly_b.dot(theta_poly)
    error = predictions - y
    gradients = (1/m) * X_poly_b.T.dot(error)
    theta_poly = theta_poly - learning_rate * gradients

# 5. Check Training Accuracy
train_pred_poly = (X_poly_b.dot(theta_poly) >= 0.5).astype(int)
acc_poly = np.mean(train_pred_poly == y)
print(f"Training Accuracy (Polynomial): {acc_poly * 100:.2f}%")

# --- เปรียบเทียบกับข้อ T10 ---
# ดูตัวเลขที่ print ออกมาเทียบกับ accuracy ของโมเดลเดิม

# ---------------------------------------------------------
# --- Prepare Submission for T12 ---

# 1. เตรียม Test Set (ต้องผ่าน function เดียวกัน)
X_test_poly = create_poly_features(X_test)

# 2. Scaling (ใช้ Mean/Std ของ Train Poly เท่านั้น!)
X_test_poly_norm = (X_test_poly - X_poly_mean) / X_poly_std

# 3. Add Bias
m_test = len(X_test_poly)
X_test_poly_b = np.c_[np.ones((m_test, 1)), X_test_poly_norm]

# 4. Predict
test_pred_poly = (X_test_poly_b.dot(theta_poly) >= 0.5).astype(int)

# 5. Save CSV
submission_poly = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": test_pred_poly
})
submission_poly.to_csv("submission_poly.csv", index=False)
print("Saved submission_poly.csv successfully!")

Original features: 4
New Polynomial features: 14
Training Accuracy (Polynomial): 81.71%
Saved submission_poly.csv successfully!


In [ ]:
# --- T13: Reduced Features (Sex & Age only) ---
import numpy as np
import pandas as pd

# 1. Prepare Data (Reduced Features)
# เลือกเฉพาะ Sex และ Age
features_reduced = ["Sex", "Age"]

X_red = train[features_reduced].values.astype(float)
y = train["Survived"].values.astype(float)

# 2. Feature Scaling (สำคัญมากเช่นเคย)
X_red_mean = X_red.mean(axis=0)
X_red_std = X_red.std(axis=0)
X_red_norm = (X_red - X_red_mean) / X_red_std

# 3. Add Bias
m = len(y)
X_red_b = np.c_[np.ones((m, 1)), X_red_norm]

# 4. Train Model (Gradient Descent)
learning_rate = 0.01
n_iterations = 2000
theta_red = np.zeros(X_red_b.shape[1])

for iteration in range(n_iterations):
    predictions = X_red_b.dot(theta_red)
    error = predictions - y
    gradients = (1/m) * X_red_b.T.dot(error)
    theta_red = theta_red - learning_rate * gradients

# 5. Check Training Accuracy
train_pred_red = (X_red_b.dot(theta_red) >= 0.5).astype(int)
accuracy_red = np.mean(train_pred_red == y)
print(f"Training Accuracy (Sex & Age only): {accuracy_red * 100:.2f}%")

# ---------------------------------------------------------
# --- Prepare Submission for T13 ---

# 1. Prepare Test Set
# (สมมติว่า test ถูก map Sex เป็น 0,1 และเติม Age เรียบร้อยแล้วจากข้อก่อนหน้า)
X_test_red = test[features_reduced].values.astype(float)

# 2. Scaling (ใช้ Mean/Std ของ Train ชุด Reduced เท่านั้น)
X_test_red_norm = (X_test_red - X_red_mean) / X_red_std

# 3. Add Bias
m_test = len(X_test_red)
X_test_red_b = np.c_[np.ones((m_test, 1)), X_test_red_norm]

# 4. Predict
test_pred_red = (X_test_red_b.dot(theta_red) >= 0.5).astype(int)

# 5. Save CSV
submission_red = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": test_pred_red
})
submission_red.to_csv("submission_reduced.csv", index=False)
print("Saved submission_reduced.csv successfully!")

Training Accuracy (Sex & Age only): 78.68%
Saved submission_reduced.csv successfully!


In [15]:

# Features & Target
features = ["Pclass", "Sex", "Age", "Embarked"]
X = train[features].values.astype(float)
y = train["Survived"].values.astype(float)

# Feature Scaling (จำเป็นสำหรับ GD, และต้องใช้เหมือนกันสำหรับ Normal Eq เพื่อเทียบ weight)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_norm = (X - X_mean) / X_std

# Add Bias
m = len(y)
X_b = np.c_[np.ones((m, 1)), X_norm]

# --- 2. Method 1: Gradient Descent (จากข้อ T10) ---
learning_rate = 0.01
n_iterations = 5000  # เพิ่มรอบหน่อยเพื่อให้มั่นใจว่า converge สุดๆ
theta_gd = np.zeros(X_b.shape[1])

for iteration in range(n_iterations):
    predictions = X_b.dot(theta_gd)
    error = predictions - y
    gradients = (1/m) * X_b.T.dot(error)
    theta_gd = theta_gd - learning_rate * gradients

# --- 3. Method 2: Matrix Inversion (Normal Equation) ---
# สูตร: theta = (X^T * X)^-1 * X^T * y
# ใช้ np.linalg.inv สำหรับหา Inverse Matrix
X_b_T = X_b.T
theta_normal = np.linalg.inv(X_b_T.dot(X_b)).dot(X_b_T).dot(y)

# --- 4. Compare Weights & Report MSE ---
print("Theta (Gradient Descent):", theta_gd)
print("Theta (Normal Equation): ", theta_normal)

# คำนวณ MSE ของความแตกต่างระหว่าง weights
mse_diff = np.mean((theta_gd - theta_normal) ** 2)

print("-" * 30)
print(f"MSE of the difference between weights: {mse_diff}")
print("-" * 30)

if mse_diff < 1e-6:
    print("Result: The weights are extremely similar.")
else:
    print("Result: There is a difference (Gradient Descent might need more iterations).")

Theta (Gradient Descent): [ 0.38383838 -0.15746036  0.2344979  -0.06576928  0.0312026 ]
Theta (Normal Equation):  [ 0.38383838 -0.15746036  0.2344979  -0.06576928  0.0312026 ]
------------------------------
MSE of the difference between weights: 1.4193085354675832e-29
------------------------------
Result: The weights are extremely similar.
